In [14]:
from student.tools import *
from student.agent import *

In [16]:
ag = MemoryAgent()
ag.run("Do not do a test run of the add memory tool")
ag.run("use the add memory tool for a test run with the key 'test'")
ag.run("use the recall memory tool for a test run with the key 'test'")

Add Memory:  ['test'] This is a test memory entry.
Recall memory:  ['test'] 
 {'26bf0d42-ecd4-4f76-9869-d966a510e78f': 'This is a test memory entry.'}


In [ ]:
import pandas as pd

def visualize_chat(chat):
    formatted = []
    for turn in chat:
        role = turn["role"].capitalize()
        if "tool_call_id" in turn:
            tool_id = turn["tool_call_id"]
            role += f" (Tool Call ID: {tool_id})"
        text = turn["content"].get("text", "")
        formatted.append((role, text))

    df = pd.DataFrame(formatted, columns=["Role", "Message"])
    df.index
    return df

visualize_chat(ag.chat.messages)


,Role,Message
0,User,Do not do a test run of the add memory tool
1,Assistant,Understood! I won't perform a test run of the ...
2,User,use the add memory tool for a test run with th...
3,Assistant (Tool Call ID: call_lb5NQmXLdBdO3jU5...,add
4,User,use the recall memory tool for a test run with...
5,Assistant (Tool Call ID: call_N4OOtgAEDdCGltA8...,{'26bf0d42-ecd4-4f76-9869-d966a510e78f': 'This...


In [ ]:
ag.chat.messages

[{'role': 'user',
  'content': {'type': 'text',
   'text': 'Do not do a test run of the add memory tool'}},
 {'role': 'assistant',
  'content': {'type': 'text',
   'text': "Understood! I won't perform a test run of the add memory tool. If you have any specific requests or questions, feel free to let me know!"}},
 {'role': 'user',
  'content': {'type': 'text',
   'text': "use the add memory tool for a test run with the key 'test'"}},
 {'role': 'assistant',
  'tool_call_id': 'call_k3gph6biko7Jc0JXj9FCK4AD',
  'content': {'type': 'text', 'text': 'add'}},
 {'role': 'user',
  'content': {'type': 'text',
   'text': "use the recall memory tool for a test run with the key 'test'"}},
 {'role': 'assistant',
  'tool_call_id': 'call_isOVUqg0qVu1gTfhVlqAcqDi',
  'content': {'type': 'text',
   'text': "{'24c2c3cd-703d-449d-8f09-696226a16a29': 'This is a test memory entry.'}"}}]

In [25]:
msg = ag.chat.additional_res['full_message'].to_dict()
if msg.get('content') is None:
    # Provide a default or skip based on your application's logic.
    msg['content'] = {'type': 'text', 'text': None}
del msg['function_call']
ag.chat.messages.append(msg)

tool_call_id = msg['tool_calls'][0]['id']


message = {
    "role": "assistant",
    "tool_call_id": tool_call_id,
    'content': {
        'type': 'text',
        'text': str("Penguin")
    }
}
ag.chat.messages.append(message)

ag.chat.messages

[{'role': 'user',
  'content': {'type': 'text',
   'text': "use the add memory tool for a test run with the key 'test'"}},
 {'role': 'assistant', 'content': {'type': 'text', 'text': ''}},
 {'content': {'type': 'text', 'text': None},
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"stimuli":["test"],"content":"This is a test memory entry."}',
     'name': 'add'},
    'id': 'call_a9gVFqi4fMI5b0Nxa3WqUYoO',
    'type': 'function'}]},
 {'role': 'assistant',
  'tool_call_id': 'call_a9gVFqi4fMI5b0Nxa3WqUYoO',
  'content': {'type': 'text', 'text': 'Penguin'}}]

In [26]:
ag.run("Think of an animal", parse=None)

'How about an elephant?'

In [27]:
ag.chat.messages

[{'role': 'user',
  'content': {'type': 'text',
   'text': "use the add memory tool for a test run with the key 'test'"}},
 {'role': 'assistant', 'content': {'type': 'text', 'text': ''}},
 {'content': {'type': 'text', 'text': None},
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"stimuli":["test"],"content":"This is a test memory entry."}',
     'name': 'add'},
    'id': 'call_a9gVFqi4fMI5b0Nxa3WqUYoO',
    'type': 'function'}]},
 {'role': 'assistant',
  'tool_call_id': 'call_a9gVFqi4fMI5b0Nxa3WqUYoO',
  'content': {'type': 'text', 'text': 'Penguin'}},
 {'role': 'user', 'content': {'type': 'text', 'text': 'Think of an animal'}},
 {'role': 'assistant',
  'content': {'type': 'text', 'text': 'How about an elephant?'}}]

In [28]:
tool_messages = ag.use_tools()
tool_messages

In [10]:
out_dict = ag.tools["add"].parse()
out_dict

{'type': 'function',
 'function': {'name': 'add',
  'description': 'This tool can be used to add a new memory item.',
  'parameters': {'type': 'object',
   'properties': {'stimuli': {'type': 'array',
     'items': {'type': 'string'},
     'description': 'Parameter stimuli'},
    'content': {'type': 'string', 'description': 'Parameter content'}},
   'required': ['stimuli', 'content']}}}

In [ ]:
def template_json_output(out_dict):
    prompt = f'''
    <output>
    You are required to output a JSON object with the following keys. If the information is not available, the value should be an empty string.
    - "content" (str): the text content of your answer.
    - "molecule" (str): the molecules of the simulation separated with "." (e.j. "butane . carbon dioxide . nitrogen").
    - "other" (str): other parameters of the simulation.
    </output>
    '''
    return prompt

print(template_json_output(out_dict))


    <output>
    You are required to output a JSON object with the following keys. If the information is not available, the value should be an empty string.
    - "content" (str): the text content of your answer.
    - "system" (str): the system of the simulation.
    - "molecule" (str): the molecules of the simulation separated with "." (e.j. "butane . carbon dioxide . nitrogen").
    - "other" (str): other parameters of the simulation.
    </output>
    
